In [1]:
add_special_tokens = False
max_length = 1024
debug = False

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig, AutoConfig

MODEL_PATH = "google/gemma-7b-it"

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(
    MODEL_PATH,
    device_map = "auto",
    trust_remote_code = True,
#     torch_dtype=torch.float16,
    # quantization_config=quantization_config,
)

# model = model.to_bettertransformer()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
train = pd.read_csv('/home/zhenlan/Desktop/Projects/LLM Prompt Recovery/Data/nbroad-v1.csv')
prompts_txt = list(train.rewrite_prompt.unique())
prompts = tokenizer.batch_encode_plus(prompts_txt,add_special_tokens=add_special_tokens,return_attention_mask=False)['input_ids']
n_prompt = len(prompts)

In [4]:
def batch_generator(token_ids, batch_size, pad_token_id):
    for i in range(0, len(token_ids), batch_size):
        input_ids = token_ids[i:i+batch_size]
        lengths = [len(input_id) for input_id in input_ids]
        max_len = max(lengths)
        input_ids_padded = np.array([np.pad(input_id, (0, max_len - len(input_id)), \
                                            constant_values=pad_token_id) for input_id in input_ids])
        yield input_ids_padded, lengths

In [5]:
out = []
for input_ids, lengths in batch_generator(prompts,4,tokenizer.pad_token_id):
    input_ids = torch.tensor(input_ids).to('cuda')
    with torch.no_grad():
        hidden_states = model(input_ids=input_ids, output_hidden_states=False,output_attentions=False,use_cache=False).last_hidden_state # (batch_size, sequence_length, hidden_size)
        # Prepare indices for gathering the last non-padded hidden states
        # Subtract 1 from lengths to get the indices of the last tokens
        last_token_indices = torch.tensor(lengths).to('cuda') - 1  # (batch_size,)
        # Expand dimensions to use for gather
        last_token_indices = last_token_indices.unsqueeze(-1).unsqueeze(-1).expand(-1, -1, hidden_states.size(-1))
        # Gather the last non-padded hidden states
        last_hidden_states = torch.gather(hidden_states, 1, last_token_indices).squeeze(1)
    out.append(last_hidden_states.cpu().numpy())

In [6]:
out = np.concatenate(out,0).astype(np.float64)

In [7]:
np.save('../Data/prompt109.npy', out)

In [9]:
out.shape

(109, 3072)